In [1]:
import numpy as np
import copy

Entering x and y.

In [2]:
while True:
    try:
        my_input = input("Eter numbers of rows and columns x, y: ")
        
        x, y = [int(i) for i in my_input.split(",")]
        
        if(x > 0 and x <= y and isinstance(x, int) == True and 
           y > 0 and y < 1000  and isinstance(y, int) == True):
            break
        else:
            raise ValueError()
            
    except ValueError:
        print('X and y should be whole numbers 0 < x <= y < 1000 and should be separated by \',\'.')

print('x is ' + str(x) + ' and y is ' + str(y) + '.')

Eter numbers of rows and columns x, y: 4
X and y should be whole numbers 0 < x <= y < 1000 and should be separated by ','.
Eter numbers of rows and columns x, y: 4,4
x is 4 and y is 4.


Creating a class Cell with a method for calculating all neighbour indexes.

In [3]:
class Cell:
    def __init__(self, colnum, rownum):
        # colnum and rownum define the position of the cell
        self.colnum = int(colnum)
        self.rownum = int(rownum)
        
    def print_info(self):
        print("colnum: " + str(self.colnum) + ", rownum: " + str(self.rownum))
    
    # returns surrounding cells indexes clockwise 
    def get_indexes_around(self):
        return [ 
            [self.colnum-1, self.rownum+1],
            [self.colnum, self.rownum+1],
            [self.colnum+1, self.rownum+1],
            [self.colnum+1, self.rownum],
            [self.colnum+1, self.rownum-1],
            [self.colnum, self.rownum-1],
            [self.colnum-1, self.rownum-1],
            [self.colnum-1, self.rownum]
        ]

Creating a class Grid. All objects of this class hold the current state of the grid, the number of it's generation and a heatmap grid with all green occurrences up to the current generation. 

In [4]:
class Grid:
    RED = 0
    GREEN = 1
    CELLS_AROUND = 8
    
    def __init__(self, x, y):
        self.x = int(x) # width
        self.y = int(y) # height
        self.grid = self.build_grid()
        self.generation = 0
        # a grid holding the sums of green occurrences
        self.grid_green_occurrences = copy.deepcopy(self.grid) 
        
    def build_grid(self):
        grid = np.empty((self.y,self.x))
        
        while True:
            try:
                for i in range(self.y):
                    grid[i] = [char for char in input("Enter red and green cells for row " + str(i) + ": ")]
                    
                    if (len(grid[i]) != self.x):
                        raise ValueError
                    for c in grid[i]:
                        if c not in(0,1):
                            raise ValueError

                grid = grid.astype(int)
                
                break
        
            except ValueError:
                print("You should enter strings with length " + str(self.x) + " containing only 0 and 1.")

        return grid
        
    def print_info(self, cell = None):
        print("x: " + str(self.x) + ", y: " + str(self.y) + ", generation: " + str(self.generation))
        print(self.grid)
        if cell is not None:
            print("[col " + str(cell.colnum) + ", row " + str(cell.rownum) + "] is surrounded by "+ str(self.get_num_surrounded_by(cell)) + ' green cells.')
            print("[col " + str(cell.colnum) + ", row " + str(cell.rownum) + "] has " + str(self.get_num_green_occurrences(cell)) + " green occurrences up to generation " + str(self.generation) + "." )
            print()
            
    # returns the number of generations the cell was green
    def get_num_green_occurrences(self, cell):
        cell_green_occurrences = self.grid_green_occurrences[cell.rownum,cell.colnum]
        return cell_green_occurrences
    
    # returns the number of surrounding cells in the specified çolour
    def get_num_surrounded_by(self, cell, colour = GREEN):
        
        valid_indexes_around = list(filter(lambda a: a[0] >= 0 and a[1] >= 0 
                                           and a[0] < self.x and a[1] < self.y, 
                                           cell.get_indexes_around()))

        surrounding_cells = [self.grid[j, i] for i, j in valid_indexes_around] # i is rownum, j is colnum
        
        # check if we search for red or green surrounding sells
        if colour == self.GREEN:
            return sum(surrounding_cells)
        else:
            return self.CELLS_AROUND - sum(surrounding_cells)
    
    # transforms the grid to a next generation grid applying the rules and updating the green occurrences heatmap
    def go_to_next_generation(self):
        self.generation += 1
        temp_grid = copy.deepcopy(self.grid)

        for colnum in range(self.x):
            for rownum in range(self.y):
                
                cell = Cell(colnum, rownum)
                
                br_green_around = self.get_num_surrounded_by(cell)
                                
                # rule 1, 2
                if self.grid[rownum][colnum] == self.RED and br_green_around in(3, 6):
                    temp_grid[rownum][colnum] = self.GREEN 

                # rule 3, 4
                if self.grid[rownum][colnum] == self.GREEN and br_green_around in(0, 1, 4, 5, 7, 8):
                    temp_grid[rownum][colnum] = self.RED

        self.grid = temp_grid
        
        self.grid_green_occurrences += self.grid
    
    def run_generations_loop(self, n):
        for i in range(n):
            self.go_to_next_generation()
        
# note that because of the task I create cells with parameters (colnum, rownum)
# but the position of the cell in the grid is specified with [rownum, colnum]

Creating an object g1 from class Grid. 

In [5]:
g1 = Grid(x, y)

Enter red and green cells for row 0: 1002
You should enter strings with length 4 containing only 0 and 1.
Enter red and green cells for row 0: 1001
Enter red and green cells for row 1: 1002
You should enter strings with length 4 containing only 0 and 1.
Enter red and green cells for row 0: 1001
Enter red and green cells for row 1: 1111
Enter red and green cells for row 2: 0100
Enter red and green cells for row 3: 1010


Entering x1, y1 and specifying the number of generations.

In [6]:
while True:
    try: 
        my_input = input("Enter the colnum, rownum and generation x1, y1, n: ")
        
        x1, y1, n =  [int(i) for i in my_input.split(",")]
        
        if(x1 >= 0 and x1 < x and isinstance(x1, int) == True and
           y1 >= 0 and y1 < y and isinstance(y1, int) == True and
           n > 0 and isinstance(n, int) == True):
            break
        else:
            raise ValueError
            
    except ValueError:
        print("X1, y1 and n should be whole numbers 0 < x1 < x, 0 < y1 < y, n > 0 and should be separated by \',\'.")

Enter the colnum, rownum and generation x1, y1, n: 2,2,15


Printing info about g1 and the cell with coordinates x1 and y1. Then running the generations loop N times and printing the info again. 

In [7]:
my_cell = Cell(x1, y1)

g1.print_info(my_cell)
g1.run_generations_loop(n)
g1.print_info(my_cell)

x: 4, y: 4, generation: 0
[[1 0 0 1]
 [1 1 1 1]
 [0 1 0 0]
 [1 0 1 0]]
[col 2, row 2] is surrounded by 5 green cells.
[col 2, row 2] has 0 green occurrences up to generation 0.

x: 4, y: 4, generation: 15
[[0 0 0 0]
 [0 0 1 1]
 [0 0 1 1]
 [0 0 0 0]]
[col 2, row 2] is surrounded by 3 green cells.
[col 2, row 2] has 14 green occurrences up to generation 15.

